In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

import os
import datetime
import itertools

# ログ出力用の関数
def printTime(tag):
    print(tag + ':', datetime.datetime.now())

printTime('Inputファイルを表示')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Inputファイルを表示: 2020-02-08 11:52:05.674267
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/data_description.txt
/kaggle/input/.ipynb_checkpoints/train-checkpoint.csv
/kaggle/input/.ipynb_checkpoints/data_description-checkpoint.txt


#### 学習データの読み込み

In [8]:
train = pd.read_csv('/kaggle/input/train.csv')
test = pd.read_csv('/kaggle/input/test.csv')

# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['SalePrice'], axis=1)
train_y = train['SalePrice']

# テストデータは特徴量のみなので、そのままでよい
test_x = test.copy()

 #### 特徴量作成

In [9]:
# 解析対象とする数値列を設定
targets_num = ['1stFlrSF', 
               '2ndFlrSF'
               'BsmtFinSF1', 
               'Fireplaces', 
               'FullBath', 
               'GarageArea', 
               'GarageCars', 
               'GarageYrBlt', 
               'GrLivArea', 
               'MasVnrArea', 
               'OverallQual', 
               'OverallCond',
               'TotalBsmtSF', 
               'TotRmsAbvGrd', 
               'YearBuilt', 
               'YearRemodAdd']

# 解析対象とするカテゴリ列を設定
targets_category = [
                    'MSSubClass',
                    'MSZoning',
                    'Street',
                    'Alley',
                    'LotShape',
                    'LandContour',
                    'Utilities',
                    'LotConfig',
                    'LandSlope',
                    'Neighborhood',
                    'Condition1',
                    'Condition2',
                    'BldgType',
                    'HouseStyle',
                    'RoofStyle',
                    'RoofMatl',
                    'Exterior1st',
                    'Exterior2nd',
                    'MasVnrType',
                    'ExterQual',
                    'ExterCond',
                    'Foundation',
                    'BsmtQual',
                    'BsmtCond',
                    'BsmtExposure',
                    'BsmtFinType1',
                    'BsmtFinType2',
                    'Heating',
                    'HeatingQC',
                    'CentralAir',
                    'Electrical',
                    'KitchenQual',
                    'Functional',
                    'FireplaceQu',
                    'GarageType',
                    'GarageFinish',
                    'GarageQual',
                    'GarageCond',
                    'PavedDrive',
                    'PoolQC',
                    'Fence',
                    'MiscFeature',
                    'SaleType',
                    'SaleCondition']

# 解析対象とする数値列とカテゴリ列の列名を連結
targets = targets_num
targets.extend(targets_category)

# 解析対象列のみを抽出
train_x = train_x.reindex(columns=targets)
test_x = test_x.reindex(columns=targets)

# Dataframeの列の型がobjectのままだとLabelEncoderでエラーするため、strに変換
# ※変換処理後にinfo()で型を確認しても、objectと表示されるが内部的にはstrに変換されている。
train_x[targets_category] = train_x[targets_category].astype(str)
test_x[targets_category] = test_x[targets_category].astype(str)

# ラベルエンコーディング用に学習データとテストデータをunion
lavel_x = train_x.append(test_x, ignore_index=True)

# カテゴリ変数をlabel encodingする
for c in targets_category:
    # 学習データとテストデータに基づいてラベルを設定
    le = LabelEncoder()
    le.fit(lavel_x[c])
    train_x[c] = le.transform(train_x[c])
    test_x[c] = le.transform(test_x[c])


#### モデル作成とバリデーション
LightGBMを使用してモデルを作成します。

In [10]:
printTime('モデルの作成開始')

# 学習データを学習データとバリデーションデータに分ける
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 特徴量と目的変数をlightgbmのデータ構造に変換する
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

# ハイパーパラメータの設定
params = {                                                                                               
    'boosting_type': 'gbdt',                                                                             
    'objective': 'regression_l2',                                                                           
    'metric': 'l2',                                                                             
    'num_leaves': 40,                                                                                    
    'learning_rate': 0.05,                                                                               
    'feature_fraction': 0.9,                                                                             
    'bagging_fraction': 0.8,                                                                             
    'bagging_freq': 5,   
    'lambda_l2': 2,
}  

# 作成する決定木の数を指定
num_round = 100

# 学習の実行
# カテゴリ変数をパラメータで指定している
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
model = lgb.train(params, 
                  lgb_train, 
                  num_boost_round=num_round,
                  categorical_feature=targets_category,
                  valid_names=['train', 'valid'], 
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=5)

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)

# バリデーションデータでのスコア(真の値の対数と予測値の対数の二乗平均平方根誤差 (RMSE))を計算する
rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))

print(f'RMSE: {rmse:.4f}')

# 予測
pred = model.predict(test_x)


# 提出用ファイルの作成
submission = pd.DataFrame({'Id': test['Id'], 'SalePrice': pred})
submission.to_csv('/kaggle/output/submission_first.csv', index=False)

printTime('モデルの作成終了')

モデルの作成開始: 2020-02-08 11:52:06.168795
[5]	train's l2: 4.53011e+09	valid's l2: 3.95589e+09
[10]	train's l2: 3.22514e+09	valid's l2: 2.80947e+09
[15]	train's l2: 2.3875e+09	valid's l2: 2.08733e+09
[20]	train's l2: 1.83396e+09	valid's l2: 1.66534e+09
[25]	train's l2: 1.45196e+09	valid's l2: 1.33341e+09
[30]	train's l2: 1.19836e+09	valid's l2: 1.14921e+09


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature', 'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'Street', 'Utilities']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[35]	train's l2: 9.92246e+08	valid's l2: 1.01288e+09
[40]	train's l2: 8.4968e+08	valid's l2: 9.47213e+08
[45]	train's l2: 7.45991e+08	valid's l2: 8.61384e+08
[50]	train's l2: 6.69215e+08	valid's l2: 8.47221e+08
[55]	train's l2: 6.12634e+08	valid's l2: 8.22516e+08
[60]	train's l2: 5.62238e+08	valid's l2: 7.93954e+08
[65]	train's l2: 5.27606e+08	valid's l2: 7.83203e+08
[70]	train's l2: 5.00891e+08	valid's l2: 7.7362e+08
[75]	train's l2: 4.77093e+08	valid's l2: 7.69235e+08
[80]	train's l2: 4.564e+08	valid's l2: 7.64986e+08
[85]	train's l2: 4.35185e+08	valid's l2: 7.70835e+08
[90]	train's l2: 4.13218e+08	valid's l2: 7.73682e+08
[95]	train's l2: 3.98128e+08	valid's l2: 7.74683e+08
[100]	train's l2: 3.79251e+08	valid's l2: 7.86789e+08
RMSE: 0.1410
モデルの作成終了: 2020-02-08 11:52:06.787009


#### モデルチューニング

In [11]:
printTime('モデルチューニング開始')

"""
# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [0.3, 0.4, 0.5]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# ループカウント変数
loopcount = 0

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

    loopcount += 1
    
    score_folds = []
    # クロスバリデーションを行う
    # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict(va_x)
        rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))
        score_folds.append(rmse)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

    printTime(f'max_depth: {max_depth}, '\
              f'min_child_weight: {min_child_weight}, '\
              f'score: {score_mean:.5f}')
    
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

"""

printTime('モデルチューニング終了')

モデルチューニング開始: 2020-02-08 11:52:06.798933
モデルチューニング終了: 2020-02-08 11:52:06.799291
